In [1]:
!pip list
!pip install --upgrade pip
# !pip uninstall torch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu129
!pip install pandas transformers tf-keras kagglehub scikit-learn hf_xet accelerate>=0.26.0

Package                 Version
----------------------- ------------
absl-py                 2.3.1
accelerate              1.11.0
asttokens               3.0.0
astunparse              1.6.3
certifi                 2025.11.12
charset-normalizer      3.4.4
colorama                0.4.6
comm                    0.2.3
contourpy               1.3.3
cycler                  0.12.1
debugpy                 1.8.17
decorator               5.2.1
executing               2.2.1
filelock                3.19.1
flatbuffers             25.9.23
fonttools               4.60.1
fsspec                  2025.9.0
gast                    0.6.0
google-pasta            0.2.0
grpcio                  1.76.0
h5py                    3.15.1
hf-xet                  1.2.0
huggingface-hub         0.36.0
idna                    3.11
ipykernel               7.1.0
ipython                 9.7.0
ipython_pygments_lexers 1.1.1
jedi                    0.19.2
Jinja2                  3.1.6
joblib                  1.5.2
jupyter_clien

In [2]:
AMOUNT_OF_REVIEWS_TO_CLASSIFY = 300

In [3]:
!nvidia-smi
import torch
print(f"GPU disponível? {torch.cuda.is_available()}")
print(f"Nome da GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Nenhuma'}")

# 1. Verifica se a GPU (CUDA) está disponível
#    Se estiver, usa "cuda:0" (a primeira GPU)
#    Se não, usa "cpu"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

print(f"Dispositivo selecionado: {device}")



Wed Nov 12 01:51:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.00                 Driver Version: 577.00         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   41C    P0             13W /   60W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device=device)  # Use GPU if available


c:\dev\python\wks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device set to use cuda:0


In [5]:
import kagglehub 
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

# Load dataset
df = pd.read_csv(path + "/IMDB Dataset.csv")    

df.head(20)


Path to dataset files: C:\Users\rafas\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [6]:
model_classification_results = []

for index, row in df.head(AMOUNT_OF_REVIEWS_TO_CLASSIFY).iterrows():
    sequence_to_classify = row['review']
    candidate_labels = ['positive', 'negative']
    result = classifier(sequence_to_classify, candidate_labels)
    print(f"Review {index+1} classification:")
    print(result['labels'][0], "with score", result['scores'][0])
    
    model_classification_results.append({
        'review': sequence_to_classify,
        'predicted_label': result['labels'][0],
        'score': result['scores'][0]
    })
    # print("\n")

# sequence_to_classify = "one day I will see the world"
# candidate_labels = ['travel', 'cooking', 'dancing']
# classifier(sequence_to_classify, candidate_labels)
#{'labels': ['travel', 'dancing', 'cooking'],
# 'scores': [0.9938651323318481, 0.0032737774308770895, 0.002861034357920289],
# 'sequence': 'one day I will see the world'}


Review 1 classification:
positive with score 0.5315369367599487
Review 2 classification:
positive with score 0.9874507784843445
Review 3 classification:
positive with score 0.9743620753288269
Review 4 classification:
negative with score 0.9753220081329346
Review 5 classification:
positive with score 0.8379409313201904
Review 6 classification:
positive with score 0.9739865660667419


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Review 7 classification:
positive with score 0.7787741422653198
Review 8 classification:
negative with score 0.9267164468765259
Review 9 classification:
negative with score 0.8084461092948914
Review 10 classification:
positive with score 0.9694986343383789
Review 11 classification:
negative with score 0.8875367045402527
Review 12 classification:
negative with score 0.5397162437438965
Review 13 classification:
negative with score 0.8973174691200256
Review 14 classification:
negative with score 0.9752565622329712
Review 15 classification:
positive with score 0.9951791167259216
Review 16 classification:
negative with score 0.9860522150993347
Review 17 classification:
negative with score 0.8709798455238342
Review 18 classification:
negative with score 0.9806430339813232
Review 19 classification:
positive with score 0.5455807447433472
Review 20 classification:
negative with score 0.9686130881309509
Review 21 classification:
positive with score 0.8618015050888062
Review 22 classification:
ne

In [7]:
for index, row in df.head(AMOUNT_OF_REVIEWS_TO_CLASSIFY).iterrows():
    predicted = model_classification_results[index]['predicted_label']
    actual_label = row['sentiment']
    if predicted != actual_label:
        print(f"Review {index+1} - Predicted: {predicted}, Actual: {actual_label}")
        

Review 17 - Predicted: negative, Actual: positive
Review 26 - Predicted: negative, Actual: positive
Review 30 - Predicted: negative, Actual: positive
Review 31 - Predicted: negative, Actual: positive
Review 32 - Predicted: negative, Actual: positive
Review 39 - Predicted: negative, Actual: positive
Review 45 - Predicted: negative, Actual: positive
Review 49 - Predicted: negative, Actual: positive
Review 52 - Predicted: negative, Actual: positive
Review 57 - Predicted: positive, Actual: negative
Review 72 - Predicted: positive, Actual: negative
Review 77 - Predicted: negative, Actual: positive
Review 117 - Predicted: negative, Actual: positive
Review 130 - Predicted: negative, Actual: positive
Review 141 - Predicted: positive, Actual: negative
Review 156 - Predicted: positive, Actual: negative
Review 164 - Predicted: positive, Actual: negative
Review 191 - Predicted: negative, Actual: positive
Review 192 - Predicted: negative, Actual: positive
Review 201 - Predicted: positive, Actual: n

In [8]:
from sklearn.metrics import classification_report

# 1. Pegar os "rótulos verdadeiros" (ground truth)
true_labels = list(df.head(AMOUNT_OF_REVIEWS_TO_CLASSIFY)['sentiment'])

# 2. Pegar os "rótulos previstos" pelo seu modelo
predicted_labels = [item['predicted_label'] for item in model_classification_results]

# 3. Gerar o relatório de classificação
print("--- MÉTRICAS DO MODELO ZERO-SHOT (Baseline) ---")
print(classification_report(true_labels, predicted_labels, target_names=['negative', 'positive']))

--- MÉTRICAS DO MODELO ZERO-SHOT (Baseline) ---
              precision    recall  f1-score   support

    negative       0.88      0.94      0.91       161
    positive       0.93      0.85      0.89       139

    accuracy                           0.90       300
   macro avg       0.90      0.90      0.90       300
weighted avg       0.90      0.90      0.90       300



In [9]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# Vamos usar um modelo mais leve para o fine-tuning, como o DistilBERT
# Ele é rápido de treinar e tem ótima performance.
MODEL_CHECKPOINT = "distilbert-base-uncased"

# Carregar o tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# --- Preparação dos Dados ---
# Vamos usar uma amostra menor para treinar rápido. 2000 exemplos são mais que suficientes.
# (Usar os 50k pode demorar muito)
sample_df = df.sample(n=2000, random_state=42) 

# Precisamos converter 'positive'/'negative' para números (0 e 1)
# O modelo não entende texto no rótulo.
sample_df['label_num'] = sample_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

# Dividir os dados em Treino e Teste
X = list(sample_df['review'])
y = list(sample_df['label_num'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenizar (converter o texto em números que o modelo entende)
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=256)

In [10]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Pega todos os 'input_ids', 'attention_mask', etc.
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Adiciona o rótulo
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Criar os datasets de treino e teste
train_dataset = IMDbDataset(train_encodings, y_train)
test_dataset = IMDbDataset(test_encodings, y_test)

In [15]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Carregar o modelo pré-treinado (com uma "cabeça" de classificação em cima)
# O num_labels=2 diz para ele se preparar para classificar entre 2 coisas (pos/neg)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,              # 2 ou 3 épocas
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,                # Configurações do otimizador
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
   # evaluation_strategy="epoch"      # Avaliar no final de cada época
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.697600
20,0.690000
30,0.683100
40,0.689900
50,0.682900
60,0.682000
70,0.670100
80,0.644000
90,0.601100
100,0.479500


TrainOutput(global_step=200, training_loss=0.5220289540290832, metrics={'train_runtime': 51.6366, 'train_samples_per_second': 61.972, 'train_steps_per_second': 3.873, 'total_flos': 211947837849600.0, 'train_loss': 0.5220289540290832, 'epoch': 2.0})

In [ ]:
import numpy as np

# 1. Fazer previsões no conjunto de teste
predictions = trainer.predict(test_dataset)

# 2. As previsões saem como "logits", precisamos do rótulo final (0 ou 1)
predicted_labels_tuned = np.argmax(predictions.predictions, axis=1)

# 3. Pegar os rótulos verdadeiros do conjunto de teste
true_labels_tuned = y_test

# 4. Gerar o relatório de classificação
print("--- MÉTRICAS DO MODELO FINE-TUNED ---")
print(classification_report(true_labels_tuned, predicted_labels_tuned, target_names=['negative', 'positive']))

--- MÉTRICAS DO MODELO FINE-TUNED ---
              precision    recall  f1-score   support

    negative       0.94      0.86      0.90       209
    positive       0.86      0.94      0.90       191

    accuracy                           0.90       400
   macro avg       0.90      0.90      0.90       400
weighted avg       0.90      0.90      0.90       400

